In [ ]:
from azure.cognitiveservices.speech import SpeechConfig
speech_config = SpeechConfig(subscription="GEBsBQkxx8vhjmrE9UtSBrP3M6uIRSex6IBs2kGsphOVAn0Epl7WJQQJ99BJACGhslBXJ3w3AAAYACOGhHgY", region="Central India")

In [ ]:
def transcribe_azure_speech(audio_path):
    """Transcribe using Azure AI Speech-to-Text with continuous recognition"""
    if not speech_config:
        raise ValueError("Azure Speech client not initialized")
    if not os.path.exists(audio_path):
        raise FileNotFoundError(f"Audio file not found: {audio_path}")
    
    print("🎤 Transcribing with Azure AI Speech-to-Text...")
    
    # Convert MP3 to WAV (Azure prefers WAV for batch processing)
    import pydub
    temp_wav = "temp_audio.wav"
    audio = pydub.AudioSegment.from_mp3(audio_path)
    audio.export(temp_wav, format="wav")
    
    # Setup audio configuration
    audio_config = AudioConfig(filename=temp_wav)
    recognizer = SpeechRecognizer(speech_config=speech_config, audio_config=audio_config)
    
    # Collect results
    transcriptions = []
    done = False

    def handle_recognized(evt):
        if evt.result.text:
            transcriptions.append(evt.result.text)
    
    def handle_done(evt):
        nonlocal done
        done = True
    
    recognizer.recognized.connect(handle_recognized)
    recognizer.session_stopped.connect(handle_done)
    recognizer.canceled.connect(handle_done)
    
    # Start continuous recognition
    recognizer.start_continuous_recognition()
    
    # Wait until done
    while not done:
        time.sleep(0.5)
    
    recognizer.stop_continuous_recognition()
    
    # Clean up temporary WAV
    if os.path.exists(temp_wav):
        os.remove(temp_wav)
    
    if not transcriptions:
        raise RuntimeError("No speech recognized or transcription failed")
    
    return " ".join(transcriptions), "Azure Speech-to-Text"

def get_transcript(audio_path):
    """Wrapper for Azure transcription"""
    return transcribe_azure_speech(audio_path)

print("✅ Transcription functions ready!")

In [ ]:
from transformers import pipeline

def make_summary(transcript):
    """Generate structured summary with OpenAI or local fallback"""
    prompt = f"""You are a professional meeting assistant. Summarize this meeting transcript in a structured format:
- **Overview**: Date (October 13, 2025), participants (if known, else generic), objectives.
- **Key Decisions**: Main decisions as bullets.
- **Action Items**: Who does what, by when (if mentioned, else note 'unspecified').
- **Next Steps**: Future plans or follow-ups.

Be concise, use bullets, and only include what's in the transcript. If unclear, note it.
Transcript: {transcript[:8000]}"""
    
    if client:
        try:
            response = client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=[{"role": "user", "content": prompt}],
                temperature=0.3
            )
            return response.choices[0].message.content, "OpenAI GPT-3.5"
        except Exception as e:
            print(f"❌ OpenAI failed: {e}. Using Hugging Face fallback...")
    
    # Fallback to Hugging Face
    summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
    chunk = transcript[:1000]
    summary = summarizer(chunk, max_length=150, min_length=50, do_sample=False)
    return summary[0]["summary_text"], "Hugging Face BART"

print("✅ Summarization function ready!")



In [ ]:
def store_results(audio_path, transcript, summary, provider):
    """Save to JSON and SQLite"""
    result = {
        "filename": os.path.basename(audio_path),
        "provider": provider,
        "transcript": transcript,
        "summary": summary,
        "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    }
    json_path = f"summary_{os.path.basename(audio_path)[:-4]}.json"
    with open(json_path, "w", encoding="utf-8") as f:
        json.dump(result, f, indent=2)
    
    db = SessionLocal()
    meeting = Meeting(filename=os.path.basename(audio_path), transcript=transcript, summary=summary)
    db.add(meeting)
    db.commit()
    db.refresh(meeting)
    db.close()
    
    return meeting.id, json_path

print("✅ Storage function ready!")


In [ ]:
import os
    print(f"Contents of directory '{os.path.dirname(audio_file_path)}': {os.listdir(os.path.dirname(audio_file_path))}")

    transcript = ""
    summary = ""
    action_items = []
    transcript_provider = "N/A"
    summary_provider = "N/A"
    action_item_provider = "N/A"
    meeting_id = "N/A"
    json_path = "N/A"

    try:
        # Step 1: Transcribe audio
        print("\n--- STEP 1: Transcribing Audio ---")
        transcript, transcript_provider = get_transcript(audio_file_path)

        if not transcript:
            print("No speech recognized. Skipping summary and action item generation.")
            summary = "No speech recognized in audio. Cannot summarize."
            action_items = ["No action items due to lack of speech."]
            summary_provider = "N/A"
            action_item_provider = "N/A"
        else:
            print("\n--- Transcription Result (first 500 chars) ---")
            print(f"Provider: {transcript_provider}")
            print(transcript[:500] + ("..." if len(transcript) > 500 else ""))

            # Step 2: Generate Summary
            print("\n--- STEP 2: Generating Summary ---")
            summary, summary_provider = make_summary(transcript)
            print(f"Provider: {summary_provider}")
            print("Summary:\n", summary)

            # Step 3: Extract Action Items
            print("\n--- STEP 3: Extracting Action Items ---")
            action_items, action_item_provider = extract_action_items(transcript)
            print(f"Provider: {action_item_provider}")
            if action_items:
                print("Action Items:")
                for item in action_items:
                    print(item)
            else:
                print("No action items found.")

        # Step 4: Store Results
        print("\n--- STEP 4: Storing Results ---")
        meeting_id, json_path = store_results(
            audio_file_path, transcript, summary, action_items,
            transcript_provider, summary_provider, action_item_provider
        )
        print(f"Saved to {json_path if json_path else 'database (error saving JSON)'} (ID: {meeting_id})")

    except RuntimeError as re:
        print(f"\n--- An expected runtime error occurred ---")
        print(f"Error: {re}")
        # Here we capture specific errors like SPXERR_INVALID_HEADER or authorization failures
        # and store a partial result indicating the failure.
        meeting_id, json_path = store_results(
            audio_file_path, transcript, f"Error during processing: {re}", ["N/A"],
            transcript_provider, "N/A", "N/A"
        )
        print(f"Failure logged with ID: {meeting_id}. Partial results saved to: {json_path}")
    except Exception as e:
        print(f"\n--- An unexpected error occurred during processing ---")
        print(f"Error: {e}")
        import traceback
        traceback.print_exc() # Print full traceback for detailed debugging
        # Store a failure record even for unexpected errors
        meeting_id, json_path = store_results(
            audio_file_path, transcript, f"Unexpected error: {e}", ["N/A"],
            transcript_provider, "N/A", "N/A"
        )
        print(f"Failure logged with ID: {meeting_id}. Partial results saved to: {json_path}")


# --- Main entry point when script is run ---
if __name__ == "__main__":
    audio_file_path = "C:/Users/Shravani Angalagunta/Downloads/Planning_Meeting.wav" # Update this if needed

    # Call the main orchestrator function
    main(audio_file_path)